In [1]:
import pyatmos

In [2]:
simul = pyatmos.Simulation()

Initializing Docker...
Pulling latest image... registry.gitlab.com/frontierdevelopmentlab/astrobiology/pyatmos
Ready.


In [3]:
simul.start()

Starting Docker container...
Container running.
Initializing simulation...
Ready.


In [4]:
simul.run(iterations=3)

Running 3 iterations
Iteration 0
Iteration 1
Iteration 2
Done.
